### Runtime test to check wether recreating the board state based on uci or
### copying and pushing is faster in python-chess

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import random
import numpy as np
from copy import deepcopy
import pandas as pd
%matplotlib inline
plt.style.use('seaborn-whitegrid')

### Note you need matplotlib >= 3.0.0 to show the histograms properly

In [ ]:
export_plots = True

In [ ]:
def game_stats_random_play(board_in, nb_games, use_mate_shortcut=True):
    half_moves_stats = np.zeros(nb_games)
    game_outcomes = []
    final_board_states = []
    nb_legal_moves = []
    
    for i in range(nb_games):
        
        # create deepcopy of the board
        board = deepcopy(board_in)
        # reset the half moves counter
        nb_half_moves = 0
        while board.is_checkmate() is False and board.can_claim_draw() is False:

            str_moves = str(board.legal_moves)

            # select a random legal move
            mv_list = list(board.legal_moves)

            # log the number of legal moves
            nb_legal_moves.append(len(mv_list))
            
            mate_mv_idx = str_moves.find('#')
            
            if  len(mv_list) > 0:
                # check if there's a possible mate on the board -> execute it
                if mate_mv_idx != -1 and use_mate_shortcut is True:
                    # find the according index of the move in the legal_moves generator list
                    mv_idx = str_moves[:mate_mv_idx].count(',')
                else:
                    # take a random legal move
                    mv_idx = np.random.randint(len(mv_list))

                # apply the move to the board
                board.push(mv_list[mv_idx])
                nb_half_moves += 1
            else:
                break

        # record the number of half moves to the list
        half_moves_stats[i] = nb_half_moves
        # log the winner of the last board state
        game_outcomes.append(board.result())
        # log the final board state to the list
        final_board_states.append(board)
        
    return half_moves_stats, game_outcomes, final_board_states, nb_legal_moves

In [ ]:
def analyze_varariants(game_variant: dict, use_mate_shortcut=True, nb_sims=10):
    for game_name in game_variant:
        game = game_variant[game_name]
        game['Game length (half moves)'], game['Game outcomes'], game['Final boards'], game['Number of legal moves'] = game_stats_random_play(game['board'], nb_sims, use_mate_shortcut)
    return game_variant

In [ ]:
game_variant = {}
#game_variant['antichess'] = {'board': chess.variant.GiveawayBoard()}
#game_variant['king of the hill'] = {'board': chess.variant.KingOfTheHillBoard()}
game_variant['chess'] = {'board': chess.Board()}
game_variant['atomic'] = {'board': chess.variant.AtomicBoard()}
game_variant['crazyhouse'] = {'board': chess.variant.CrazyhouseBoard()}
game_variant = analyze_varariants(game_variant, use_mate_shortcut=True, nb_sims=50)

In [ ]:
def apply_def(filename):
    plt.legend(frameon=True)
    if export_plots is True:
        plt.savefig('./plots/%s.png'%filename, bbox='tight')
        plt.savefig('./plots/%s.pdf'%filename, bbox='tight')
    plt.show()

In [ ]:
# select all  the stats
stats = list(game_variant[list(game_variant.keys())[0]].keys())

for stat in stats:
    
    if 'board' not in stat:
        # show a new histogram plot
        plt.figure()
        for variant in game_variant:
            game_variant[variant][stat] = plt.hist(game_variant[variant][stat], alpha=1/len(game_variant), label=variant, stacked=True)
            plt.title(stat)
        filename = stat.replace(' ', '_')
        apply_def(filename)